In [8]:
import csv
import tweepy
import pandas as pd
import spacy
from spacy.lang.id import Indonesian
import unicodedata
from unidecode import unidecode
import html
import re
import emoji
from googletrans import Translator
import json

In [4]:
# Auth
auth = tweepy.OAuthHandler('ynr485xkIGMh5m3RwEhhiK3Y4', 'xooRDYC235j8FTkeZe3s3NcmvzMqndy264RXAVwIQvvZsksbGk')
auth.set_access_token('1226612648-kimIpngGwgx3tceLibaIjNpVUAyfjLm2DjkwVa5', 'hknzO1NV885ghvSsymN5ihJArAqj1tLZZXlitXrJcM1ke')

api = tweepy.API(auth)

In [5]:
# Crawl
venom_jkt_tweets = tweepy.Cursor(api.search, q='fantastic beast', geocode="-6.265750,106.781821,150km").items()

In [20]:
#CSV File
with open('fb_jkt_tweetss.csv', 'w', encoding="utf-8") as csv_tweet:

    fieldnames_tweet = ['id', 'user_id', 'created_at', 'text']
    writer_tweet = csv.DictWriter(csv_tweet, fieldnames=fieldnames_tweet)
    writer_tweet.writeheader()

    for tweet in venom_jkt_tweets:
        if "http" not in tweet.text and "@" not in tweet.text:
            #line = re.sub("[^A-Za-z]", " ", tweet.text)
            #user_id = tweet.user.id
            writer_tweet.writerow({
                'id': tweet.id, 
                'user_id': tweet.user.screen_name,
                'created_at': tweet.created_at, 
                'text': tweet.text
            })      
        

In [3]:
#load
fb_jkt = pd.read_csv('fb_jkt_tweets.csv')
# fb_text = fb_jkt['text']
fb_jkt.head()

,id,user_id,created_at,text
0,1066193963265085440,cessphoebe,2018-11-24 04:57:04,Begitu gajian rasanya pengen nonton fantastic ...
1,1066039906655338496,aulazack,2018-11-23 18:44:54,"Saran film yg seru? Model"" fantastic beast kal..."
2,1066000870058278912,setchan89,2018-11-23 16:09:47,Abis nonton fantastic beast trus PAAN SIH INI
3,1065991814400770051,niekoap,2018-11-23 15:33:48,Habis nonton fantastic beast. Terus langsung o...
4,1065989076019376128,gebifelina,2018-11-23 15:22:55,Baru sempet nonton fantastic beast. Satu pikir...


In [9]:
#Preprocessing

nlp_spacy_id = Indonesian()
pd.set_option('max_colwidth', 100)

def normalize_slang_words(tokenized_sentence):
    slang_word_dict = json.loads(open('slang_word_dict.txt', 'r').read())
    for index in range(len(tokenized_sentence)):
        for key, value in slang_word_dict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return tokenized_sentence

def normalize_repeated_char(tokenized_sentence):
    alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
                'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 
                'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
                'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    tokenized_sentence = ' '.join(tokenized_sentence)
    for i in range(len(alphabet)):
        charac_long = 5
        while charac_long >= 3:
            char = alphabet[i] * charac_long
            tokenized_sentence = tokenized_sentence.replace(char, alphabet[i])
            charac_long -= 1        
    tokenized_sentence = tokenized_sentence.split()    
    return tokenized_sentence

def preprocessing(comment):
    
    comment = comment.lower() #lower
    tokenS_id = nlp_spacy_id(comment) #Token
    ContentnoRepeat = normalize_repeated_char(token.text for token in tokenS_id) #repeated char
    ContentnoSlang = normalize_slang_words(ContentnoRepeat) #Slang
    ContentnoHTML = html.unescape(str(ContentnoSlang)) #html tag 
    ContentnoEmoji = emoji.demojize(ContentnoHTML) #Emoji
    ContentnoPunct = re.sub('[^ a-zA-Z0-9]', ' ',ContentnoEmoji) #Punctuation
    
#     stopfactory = StopWordRemoverFactory() #Stop word
#     stopword = stopfactory.create_stop_word_remover()
#     stopRemove = stopword.remove(ContentnoPunct)

    content = list()
    content.append(ContentnoPunct)
 
    return " ".join(content)

fb_jkt['Clean'] = fb_jkt['text'].apply(preprocessing)
fb_jkt.head(20)

,id,user_id,created_at,text,Clean
0,1066193963265085440,cessphoebe,2018-11-24 04:57:04,Begitu gajian rasanya pengen nonton fantastic beast-suzanna-wreck it ralph langsung sehari :'),begitu gajian rasanya ingin nonton fantastic beast suzanna w...
1,1066039906655338496,aulazack,2018-11-23 18:44:54,"Saran film yg seru? Model"" fantastic beast kalo bisa",saran film yang seru model fantastic beast kalau bisa
2,1066000870058278912,setchan89,2018-11-23 16:09:47,Abis nonton fantastic beast trus PAAN SIH INI,habis nonton fantastic beast terus paan sih ini
3,1065991814400770051,niekoap,2018-11-23 15:33:48,Habis nonton fantastic beast. Terus langsung open discussion dibahas sampe malem. Ngalahin disku...,habis nonton fantastic beast terus langsung open discussion dibah...
4,1065989076019376128,gebifelina,2018-11-23 15:22:55,"Baru sempet nonton fantastic beast. Satu pikiran gua daritadi "" ini rasa penasaran gua harus ter...",baru sempat nonton fantastic beast satu pikiran saya daritadi ...
5,1065986839171878912,annisa_rianti,2018-11-23 15:14:02,Fix pemeran Fantastic Beast udah dipantau sejak lama sama tim. Callum Turner pernah main sama Da...,fix pemeran fantastic beast sudah dipantau sejak lama sama tim ...
6,1065984642497441793,shittymuc,2018-11-23 15:05:18,Fantastic Beast 2 didn't look so fantastic like the 1st one. But still loved Eddie ❤️ #fridaynig...,fantastic beast 2 didn t look so fantastic like the 1st one ...
7,1065970037813432321,abrarkadabra,2018-11-23 14:07:16,Ralph breaks the internet &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt; Fantastic Beast : Rise of the silver ...,ralph breaks the internet gt fantastic beast rise ...
8,1065969011962216450,ineedspacexx,2018-11-23 14:03:11,setelah nonton fantastic beast saya penasaran nanti kelanjutannya gimana.. nunggu bertahun tahun...,setelah nonton fantastic beast saya penasaran nanti kelanjutannya gima...
9,1065946976129212416,AristaRiris1,2018-11-23 12:35:38,"Ralph udh main hari ini, belum nonton Fantastic beast.. belum nonton Yg lain lain lain :((( 😣",ralph sudah main hari ini belum nonton fantastic beast b...


In [11]:
def translating(indo_blob):
    translator = Translator()
    
    blob = translator.translate(indo_blob, dest='en').text 
    contentTrans = list()
    contentTrans.append(blob)
    
    return " ".join(contentTrans)

fb_jkt['Translate'] = fb_jkt['Clean'].apply(translating)
fb_jkt.head(20)

,id,user_id,created_at,text,Clean,Translate
0,1066193963265085440,cessphoebe,2018-11-24 04:57:04,Begitu gajian rasanya pengen nonton fantastic beast-suzanna-wreck it ralph langsung sehari :'),begitu gajian rasanya ingin nonton fantastic beast suzanna w...,once the payday feels like watching fantastic beast and wrapping it instantly a day
1,1066039906655338496,aulazack,2018-11-23 18:44:54,"Saran film yg seru? Model"" fantastic beast kalo bisa",saran film yang seru model fantastic beast kalau bisa,exciting movie advice on a fantastic beast if possible
2,1066000870058278912,setchan89,2018-11-23 16:09:47,Abis nonton fantastic beast trus PAAN SIH INI,habis nonton fantastic beast terus paan sih ini,"after watching fantastic beast, then this guy"
3,1065991814400770051,niekoap,2018-11-23 15:33:48,Habis nonton fantastic beast. Terus langsung open discussion dibahas sampe malem. Ngalahin disku...,habis nonton fantastic beast terus langsung open discussion dibah...,"after watching fantastic beast and then open the discussion immediately, it was discussed until ..."
4,1065989076019376128,gebifelina,2018-11-23 15:22:55,"Baru sempet nonton fantastic beast. Satu pikiran gua daritadi "" ini rasa penasaran gua harus ter...",baru sempat nonton fantastic beast satu pikiran saya daritadi ...,"I had just watched my fantastic beast one thought from this, so my curiosity had to be answered ..."
5,1065986839171878912,annisa_rianti,2018-11-23 15:14:02,Fix pemeran Fantastic Beast udah dipantau sejak lama sama tim. Callum Turner pernah main sama Da...,fix pemeran fantastic beast sudah dipantau sejak lama sama tim ...,the fix for the fantastic beast cast has been monitored for a long time with the callum turner t...
6,1065984642497441793,shittymuc,2018-11-23 15:05:18,Fantastic Beast 2 didn't look so fantastic like the 1st one. But still loved Eddie ❤️ #fridaynig...,fantastic beast 2 didn t look so fantastic like the 1st one ...,fantastic beast 2 didn t look so fantastic like the 1st one ...
7,1065970037813432321,abrarkadabra,2018-11-23 14:07:16,Ralph breaks the internet &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt; Fantastic Beast : Rise of the silver ...,ralph breaks the internet gt fantastic beast rise ...,ralph breaks the internet gt fantastic beast rise ...
8,1065969011962216450,ineedspacexx,2018-11-23 14:03:11,setelah nonton fantastic beast saya penasaran nanti kelanjutannya gimana.. nunggu bertahun tahun...,setelah nonton fantastic beast saya penasaran nanti kelanjutannya gima...,"after watching fantastic beast, I was curious about the follow-up, how about waiting for more years"
9,1065946976129212416,AristaRiris1,2018-11-23 12:35:38,"Ralph udh main hari ini, belum nonton Fantastic beast.. belum nonton Yg lain lain lain :((( 😣",ralph sudah main hari ini belum nonton fantastic beast b...,Ralph has been playing today but hasn't watched Fantastic Beast yet watching other others persev...


In [21]:
#bp_surabaya_tweets = tweepy.Cursor(api.search, q='venom', geocode="-7.2574719,112.75208829999997,50km").items()

In [25]:
# with open('bp_surabaya_tweets.csv', 'w', encoding="utf-8") as csv_tweet, open('bp_users_surabaya.csv', 'w', encoding="utf-8") as csv_user:
#     fieldnames_tweet = ['id', 'text', 'user_id', 'created_at']
#     writer_tweet = csv.DictWriter(csv_tweet, fieldnames=fieldnames_tweet)
#     writer_tweet.writeheader()

#     fieldnames_user = ['id', 'name', 'screen_name', 'location', 'followers_count', 'friends_count']
#     writer_user = csv.DictWriter(csv_user, fieldnames=fieldnames_user)
#     writer_user.writeheader()

#     for tweet in bp_surabaya_tweets:
#         if not tweet.retweeted and ('RT @' not in tweet.text):
#             writer_tweet.writerow({
#                 'id': tweet.id, 
#                 'text': tweet.text, 
#                 'user_id': tweet.user.id,
#                 'created_at': tweet.created_at
#             })


#             writer_user.writerow({
#                 'id': tweet.user.id, 
#                 'name': tweet.user.name, 
#                 'screen_name': tweet.user.screen_name,
#                 'location': tweet.user.location,
#                 'followers_count': tweet.user.followers_count,
#                 'friends_count': tweet.user.friends_count
#             })        
        

In [34]:
# bp_bandung_tweets = tweepy.Cursor(api.search, q='venom', geocode="-6.9174639,107.61912280000001,50km").items()

In [35]:
# with open('bp_bandung_tweets.csv', 'w', encoding="utf-8") as csv_tweet, open('bp_users_bandung.csv', 'w', encoding="utf-8") as csv_user:
#     fieldnames_tweet = ['id', 'text', 'user_id', 'created_at']
#     writer_tweet = csv.DictWriter(csv_tweet, fieldnames=fieldnames_tweet)
#     writer_tweet.writeheader()

#     fieldnames_user = ['id', 'name', 'screen_name', 'location', 'followers_count', 'friends_count']
#     writer_user = csv.DictWriter(csv_user, fieldnames=fieldnames_user)
#     writer_user.writeheader()

#     for tweet in bp_bandung_tweets:
#         if not tweet.retweeted and ('RT @' not in tweet.text):
#             writer_tweet.writerow({
#                 'id': tweet.id, 
#                 'text': tweet.text, 
#                 'user_id': tweet.user.id,
#                 'created_at': tweet.created_at
#             })


#             writer_user.writerow({
#                 'id': tweet.user.id, 
#                 'name': tweet.user.name, 
#                 'screen_name': tweet.user.screen_name,
#                 'location': tweet.user.location,
#                 'followers_count': tweet.user.followers_count,
#                 'friends_count': tweet.user.friends_count
#             })        
        

In [54]:
# auth = tweepy.OAuthHandler('ynr485xkIGMh5m3RwEhhiK3Y4', 'xooRDYC235j8FTkeZe3s3NcmvzMqndy264RXAVwIQvvZsksbGk')
# auth.set_access_token('1226612648-kimIpngGwgx3tceLibaIjNpVUAyfjLm2DjkwVa5', 'hknzO1NV885ghvSsymN5ihJArAqj1tLZZXlitXrJcM1ke')

# api = tweepy.API(auth)

In [58]:
# #Getting Geo ID for USA
# places = api.geo_search(lat="-6.265750",long="106.781821", accuracy="100000m")

# #Copy USA id
# place_id = places[0].id
# print('JKT id is: ',place_id)

JKT id is:  e92102d30403f7cf


In [59]:
# #This is what we are searching for
# #We can restrict the location of tweets using place:id 
# #We can search for multiple phrases using OR
# searchQuery = 'place:e92102d30403f7cf "venom" OR "film venom" OR "nonton venom" OR "nonton film venom"'

In [60]:
# #Maximum number of tweets we want to collect 
# maxTweets = 1000000

# #The twitter Search API allows up to 100 tweets per query
# tweetsPerQry = 100

In [64]:
# tweetCount = 0

# #Open a text file to save the tweets to
# with open('PoGo_USA_Tutorial.json', 'w') as f:

#     #Tell the Cursor method that we want to use the Search API (api.search)
#     #Also tell Cursor our query, and the maximum number of tweets to return
#     for tweet in tweepy.Cursor(api.search,q=searchQuery).items(maxTweets) :         

#         #Verify the tweet has place info before writing (It should, if it got past our place filter)
#         if tweet.place is not None:
            
#             #Write the JSON format to the text file, and add one to the number of tweets we've collected
#             f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
#             tweetCount += 1

#     #Display how many tweets we have collected
#     print("Downloaded {0} tweets".format(tweetCount))

Downloaded 2 tweets


In [66]:
# #Let's look at what we are actually printing to text  (if you don't know what JSON is: http://json.org/)
# tweet._json

In [6]:
# searchQuery = '"venom" OR "film venom" OR "nonton venom" OR "nonton film venom"'

In [38]:
# bp_jogja_tweets = tweepy.Cursor(api.search, q='venom', geocode="-7.7955798,110.36948959999995,50km").items()

In [39]:
# with open('bp_jogja_tweets2.csv', 'w', encoding='utf-8') as csv_tweet, open('bp_users_jogja.csv', 'w', encoding='utf-8') as csv_user:
#     fieldnames_tweet = ['id', 'text', 'user_id', 'created_at']
#     writer_tweet = csv.DictWriter(csv_tweet, fieldnames=fieldnames_tweet)
#     writer_tweet.writeheader()

#     fieldnames_user = ['id', 'name', 'screen_name', 'location', 'followers_count', 'friends_count']
#     writer_user = csv.DictWriter(csv_user, fieldnames=fieldnames_user)
#     writer_user.writeheader()

#     for tweet in bp_jogja_tweets:
#         if not tweet.retweeted and ('RT @' not in tweet.text):
#             writer_tweet.writerow({
#                 'id': tweet.id, 
#                 'text': tweet.text, 
#                 'user_id': tweet.user.id,
#                 'created_at': tweet.created_at
#             })

#             writer_user.writerow({
#                 'id': tweet.user.id, 
#                 'name': tweet.user.name, 
#                 'screen_name': tweet.user.screen_name,
#                 'location': tweet.user.location,
#                 'followers_count': tweet.user.followers_count,
#                 'friends_count': tweet.user.friends_count
#             })        
        